## MMAT (MULTI-MODAL-AGENT-TRANSLATION)

Partager


Vous avez dit :
Fait moi en markdown un text qui explique les étapes pour avoir accès a AWS. Voici les élements à prendre ne compte
markdown 
### MISE EN ROUTE
#### Installation
https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
#### Configuration (aws configure)
https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html
ChatGPT a dit :
markdown
Copier
Modifier
# Guide d'accès à AWS

## MISE EN ROUTE

### Installation de l'AWS CLI

Pour interagir avec les services AWS depuis votre terminal, il est recommandé d'installer l'AWS Command Line Interface (CLI). Suivez les étapes ci-dessous pour installer l'AWS CLI sur votre système :

1. **Télécharger l'installateur** :

   - **Linux x86 (64-bit)** :

     ```bash
     curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
     ```

   - **macOS** :

     ```bash
     curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg"
     ```

   - **Windows** :

     Téléchargez le fichier MSI depuis le site officiel d'AWS.

2. **Installer l'AWS CLI** :

   - **Linux** :

     ```bash
     unzip awscliv2.zip
     sudo ./aws/install
     ```

   - **macOS** :

     ```bash
     sudo installer -pkg AWSCLIV2.pkg -target /
     ```

   - **Windows** :

     Exécutez le fichier MSI téléchargé et suivez les instructions à l'écran.

Pour des instructions détaillées et les dernières mises à jour, consultez la documentation officielle : [Installation de l'AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html).

### Configuration de l'AWS CLI

Après l'installation, configurez l'AWS CLI avec vos informations d'identification AWS :

1. **Exécutez la commande de configuration** :

   ```bash
   aws configure
### Sourcs: 
https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
#### Configuration (aws configure)
https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html

In [13]:
import boto3
import botocore
import chainlit as cl
from langchain_aws import BedrockEmbeddings, ChatBedrock
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from typing import Any, Dict, List

In [14]:
# intiatilization of the bedrock client
client_config = botocore.config.Config(max_pool_connections=100)
bedrock_client = boto3.client(
            "bedrock-runtime", region_name="us-east-1", config=client_config
)

# intialization of the chat model
llm = ChatBedrock(
            client=None,
            model_id="us.meta.llama3-3-70b-instruct-v1:0",
            region_name="us-west-2",
            model_kwargs={"temperature":0},
        )
# intialized the chat prompt
embedding = BedrockEmbeddings(
            client=bedrock_client,
            model_id="amazon.titan-embed-text-v2:0",
            region_name="us-east-1",
        )

2025-02-18 16:44:46 - Found credentials in shared credentials file: ~/.aws/credentials


In [15]:



class TranslatorAgent:
    """
        this class is an example of Translation agent that uses the bedrock model to 
        translate text to a target language.
        It support normal text and JSON object translation.
    """
    def __init__(self, model: str, target_language: str='french'):
        self.model = model
        self.target_language = target_language

    def prompt_template_json_to_json(self):
        return """
            # TRANSLATION RULES
            Your and agent desitinated to translate text to a target language.
            - Do the translation by using the appropriate term in Tender Offer document
            - Ensure the translation is precise and conveys the exact meaning of the original document.
            - Maintain the original formatting and structure of the document to ensure clarity and consistency.
            - Ensure the translation complies with any specific requirements mentioned in the tender documents.

            # INSTRUCTION
            - You will be asked to translate the a \`\`\`JSON\`\`\`\` object to the target language. 
                respect the \`\`\`JSON\`\`\`\`  structure and the special characters.
            - The target language will be mentioned in the prompt.
            - Respect the special characters and formatting of the text.
            - Maintain the original formatting and structure of the document to ensure clarity and consistency.
            - Ensure the translation complies with any specific requirements mentioned in the tender documents
            - Do not add any additional information to the translation. Only translate the text provided. 
            - Do not comment on the text or provide any additional information.
            - If the content is in the target language, do not translate it back to english.

            # TRARGET LANGUAGE
            ----------------------------------------------------------------
            {target_language}
            ----------------------------------------------------------------

            # TEXT TO TRANSLATE
            ----------------------------------------------------------------
            {text_to_translate}
            ----------------------------------------------------------------
            
            # FINAL TRANSLATION
            - Do not add introduction in your response or any additional information, 
            no phrase like 'Voici', 'Voila', 'I think', 'I believe', 'I suggest', etc. 'Voici la traduction' or 'Voilà' are also not allowed.

            # TASK
            Translate the content \`\`\`JSON\`\`\`\`  object in english into  the target language, 
            while preserving the \`\`\`JSON\`\`\`\` object structure and special characters.

        """

    def prompt_template_text_to_text(self):
        return """
            # TRANSLATION RULES
            Your and agent desitinated to translate text to a target language.
            - Ensure the translation is precise and conveys the exact meaning of the text.
            - Ensure the translation precise
            - A good translation is accueate and precise.
            - The feild of translation concern the 'tender offer' document.


            # INSTRUCTION

            - The target language will be mentioned in the prompt.
            - Respect the special characters and formatting of the text.
            - Maintain the original formatting and structure of the document to ensure clarity and consistency.
            - Ensure the translation complies with any specific requirements mentioned in the tender documents
            - Do not add any additional information to the translation. Only translate the text provided. 
            - Do not comment on the text or provide any additional information.
            - If the content is in the target language, do not translate it back to english.

            # TRARGET LANGUAGE
            ----------------------------------------------------------------
            {target_language}
            ----------------------------------------------------------------

            # TEXT TO TRANSLATE
            ----------------------------------------------------------------
            {text_to_translate}
            ----------------------------------------------------------------
            
            # FINAL TRANSLATION
            - Do not add introduction in your response or any additional information, 
            no phrase like 'Voici', 'Voila', 'I think', 'I believe', 'I suggest', etc. 'Voici la traduction' or 'Voilà' are also not allowed.
            - Do a brute translation of the text to the target language.

            # TASK
            Translate the conten in english into  the target language, 


        """

    def translate(self, content, parser=StrOutputParser()) -> str:
        
        """
        Translate the given content to the target language.

        Parameters:
        content (str or dict or list): The text or data to be translated. If a dictionary is provided, it will be converted to a list.
        parser (StrOutputParser, optional): An instance of StrOutputParser to process the output. Defaults to StrOutputParser().

        Returns:
        str: The translated text in the target language.

        If the content is empty, an empty string is returned.
        """
        if content != "":
            print("Translating the text to the target language ...")
            if isinstance(content, dict):
                content = [content]

            if isinstance(content, list):
                prompt_template = self.prompt_template_json_to_json()
            else:
                prompt_template = self.prompt_template_text_to_text()
            
            template = ChatPromptTemplate([
                ("system", prompt_template), 
                ("user", "{text_to_translate}")
            ]) 
            input_dict = {
                "target_language": self.target_language,
                "text_to_translate": content
            }

            chain = template | self.model | parser
            final_answer = chain.invoke(input_dict)
            return final_answer
        return ""

In [16]:
translatior_agent = TranslatorAgent(llm, 'french')


translatior_agent.translate(
"""
    Benjamin F. McAdoo (1920–1981) was an American architect mainly\n
    active in the Seattle area. Born in Pasadena, California, he was\n
    inspired to study architecture by a mechanical-drawing class\n
    and the work of Paul R. Williams.
""")

Translating the text to the target language ...


"Benjamin F. McAdoo (1920-1981) était un architecte américain principalement actif dans la région de Seattle. Né à Pasadena, en Californie, il a été inspiré à étudier l'architecture par un cours de dessin mécanique et l'œuvre de Paul R. Williams."

In [17]:
json_content = {
"type_of_classification": "cctp",
"final_answer_classification": "Yes"
}
translatior_agent.translate(json_content)

Translating the text to the target language ...


"[{'type_of_classification': 'cctp', 'final_answer_classification': 'Oui'}]"